In [12]:
import json
from us import states
from bson.json_util import dumps
from pymongo import MongoClient

#Connect to Mongo DB
myclient = MongoClient("mongodb://localhost:27017/")
  
#Census Data
db = myclient["CensusData"]

collections = db.list_collection_names()
print ("collections:", collections, "\n")


collections: ['counties_geocoding', 'states', 'counties', 'census_datadump', 'popincome_data', 'merged_final', 'insurance_data', 'counties_sub', 'deathrate_data'] 



In [19]:
total_population_json = db.popincome_data.aggregate(
    [{"$group": { "_id": "null", "TotalPop": { "$sum": "$Population"}}}])
# Get list of counties
for value in total_population_json:
    print(int(value["TotalPop"]))


984131163


In [2]:
popincome_data = db.popincome_data.find()

counties_collected = []
# List to store the outputed dictionaries
counties_data = []
for popincome in popincome_data:

    insurance_data = db.insurance_data.find(({"state": popincome["state"],
                                          "county": popincome["county"],
                                          "time":popincome["Year"]}),{'_id': False})

    #If this is the first time encountering a particular county, creates a new dictionary for it, stores data
    if popincome["Name"] not in counties_collected:
        counties_collected.append(popincome["Name"])
        json = {}
        json["Name"] = popincome["Name"],
        json["state"] = popincome["state"]
        json["county"] = popincome["county"]
        information = {}
        information["Population"] = popincome["Population"],
        information["Median Age"] = popincome["Median Age"]
        information["Per Capita Income"] = popincome["Per Capita Income"]
        information["Poverty Count"] = popincome["Poverty Count"]
        for insurance_info in insurance_data:
            information["Insured"] = insurance_data["NIC_PT"]
            information["Uninsured"] = insurance_data["NUI_PT"]
        # information['Educational Attainment'] = popincome['Educational Attainment']
        json[f'Y{popincome["Year"]}'] = information
        # json[popincome["Year"]] = information
        counties_data.append(json)
        
    # If it has arleady encounterd the county, appends the standing entry and stores the new year's data
    else:
        location = counties_collected.index(popincome["Name"])
        json = counties_data[location]
        information = {}
        information["Population"] = popincome["Population"],
        information["Median Age"] = popincome["Median Age"]
        information["Per Capita Income"] = popincome["Per Capita Income"]
        information["Poverty Count"] = popincome["Poverty Count"]
        for insurance_info in insurance_data:
            information["Insured"] = insurance_data["NIC_PT"]
            information["Uninsured"] = insurance_data["NUI_PT"]
        # information['Educational Attainment'] = popincome['Educational Attainment']
        json[f'Y{popincome["Year"]}'] = information
        # json[popincome["Year"]] = information

In [7]:
print(counties_data[3000])

{'Name': ('Lamar County, Texas',), 'state': '48', 'county': '277', 'Y2018': {'Population': (49532.0,), 'Median Age': 40.8, 'Per Capita Income': 23833.0, 'Poverty Count': 8685.0}, 'Y2019': {'Population': (49611.0,), 'Median Age': 40.7, 'Per Capita Income': 25038.0, 'Poverty Count': 8530.0}, 'Y2020': {'Population': (49705.0,), 'Median Age': 40.7, 'Per Capita Income': 24826.0, 'Poverty Count': 9020.0}}


In [8]:
county_ids = []
for county in counties_data:
    county_id = county["state"]+county["county"]
    county_ids.append(county_id)

In [11]:
counties_list = db.counties.find()
counties_geojson_data = db.counties_geocoding.find({},{"_id":False})

counties_geojson_final=[]

failed_county_lookup=[]
for county_geojson in counties_geojson_data:
    county_geojson_local = dumps(county_geojson)
    lookup_county = county_geojson["properties"]['STATE'] + county_geojson["properties"]['COUNTY']
    try:
        #Attempts to locate any county which has the same county_id in the censusJson List
        location = county_ids.index(str(lookup_county))
        matching_county_data = counties_data[location]
        county_geojson_local["properties"]["Y2018"] = matching_county_data["Y2018"]
        county_geojson_local["properties"]["Y2019"] = matching_county_data["Y2019"]
        county_geojson_local["properties"]["Y2020"] = matching_county_data["Y2020"]
        counties_geojson_final.append(county_geojson)
    except Exception:
        #If it cannot locate it stores the id for this county in the fail list
        failed_county_lookup.append(lookup_county)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "d:\Anaconda\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "d:\Anaconda\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [18]:
for county_geojson in counties_geojson_final:
    #Generate the countyID as above for each GeoJson
    county_geojson_check = county_geojson["properties"]['STATE'] + county_geojson["properties"]['COUNTY']
    if county_geojson_check in failed_county_lookup:
        counties_geojson_final.remove(county_geojson)

In [19]:
push_to_mongo = db.merged_final
update_result = push_to_mongo.insert_many(counties_geojson_final)
print(update_result.inserted_ids)


TypeError: documents must be a non-empty list

In [ ]:
# county_geocoding_list = []
# # for each county build a json with state FIPS id
# for state in states_l:
#     cursor_geocoding = db.counties_geocoding.find({"properties.STATE": states_l["STATENAME"],
#                                                       "properties.COUNTY": states_l["STATEFP"]},{'_id': False})
#     # query_geocoding = db.counties_geocoding.find(
#     #                 {"$and": [{ "properties" : { "$elemMatch" :{ "STATE": state_fips.fips}} },
#     #                 { "properties" : { "$elemMatch" :{ "COUNTY": county["county_fips"]}}d
#     #                 }]})
#     for county_geocoding in cursor_geocoding:
#         county_geocoding_list[0].append(county_geocoding)
#     print (county_geocoding_list)
# #     state_fips = us.states[county_geo_record["state_id"]].fips
# #     print(county_geo_record["state_id"],state_fips)